In [1]:
import pyro
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyro.optim as optim
import pyro.distributions as dist
from torch.distributions import constraints
from tqdm import tqdm_notebook as tqdm
import seaborn as sns
from matplotlib import animation, rc
from IPython.display import HTML
import torch.nn as nn
from functools import partial
import pandas as pd
from pyro.contrib.autoguide import AutoDiagonalNormal, AutoMultivariateNormal
from pyro.infer import EmpiricalMarginal, SVI, Trace_ELBO, TracePredictive
from scipy.stats import norm
from sklearn.datasets import load_breast_cancer, load_iris, load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize, KBinsDiscretizer
from sklearn.naive_bayes import GaussianNB

In [271]:
pyro.set_rng_seed(1)
pyro.enable_validation(True)
torch.set_default_tensor_type('torch.DoubleTensor')


In [279]:


def model1(data_x, data_y):  
    pk_prior = pyro.distributions.Dirichlet(torch.ones(class_amount, ))
    pk_sample = pyro.sample('pk_sample', pk_prior)

    with pyro.plate('class_amount', size=class_amount):
        with pyro.plate('atr_amount', size=atr_amount):
            prior_loc =  pyro.distributions.Normal(loc=torch.ones(atr_amount, class_amount)*2, scale=torch.ones(atr_amount, class_amount)*2)
            prior_scale = pyro.distributions.Normal(torch.ones(atr_amount, class_amount)*0.5, torch.ones(atr_amount, class_amount))
            loc = pyro.sample('mean_sample', prior_loc)
            scale = pyro.sample('std_sample', prior_scale)

    lhat = bayes_probs(data_x, loc, scales, pk_sample)
    #print(lhat)
    with pyro.plate('plate', len(data_y)):
        cat = pyro.distributions.Categorical(probs=lhat)
        pyro.sample("obs", cat, obs=data_y) 

def guide1(data_x, data_y):
    
    
    
    param_loc_loc = pyro.param('loc_loc', torch.ones(atr_amount, class_amount)*1.5)
    param_loc_sca = pyro.param('loc_sca', torch.ones(atr_amount, class_amount), 
                               constraint=constraints.positive)
    
    param_sca_loc = pyro.param('sca_loc', torch.ones(atr_amount, class_amount)*0.6, 
                               constraint=constraints.positive)
    param_sca_sca = pyro.param('sca_sca', torch.ones(atr_amount, class_amount)*0.5, 
                               constraint=constraints.positive)
            
    class_prob = pyro.param("param_class_prob", torch.ones(class_amount, ), 
                            constraint=constraints.interval(0.01, 1000))

    pk_prior = pyro.distributions.Dirichlet(class_prob)
    pk_sample = pyro.sample('pk_sample', pk_prior)
    
    with pyro.plate('class_amount', size=class_amount):
        with pyro.plate('atr_amount', size=atr_amount):
            loc = pyro.sample('mean_sample', pyro.distributions.Normal(param_loc_loc, param_loc_sca))
            
            scale = pyro.sample('std_sample', pyro.distributions.Normal(param_sca_loc, param_sca_sca))
            
            #param_atr_loc.append(loc)
            #param_atr_sca.append(scale)
        #param_loc.append(param_atr_loc)
        #param_sca.append(param_atr_sca)
    return loc, scale, pk_sample

In [276]:

locs = torch.tensor([[4., 3., 2.], [3., 3., 4.], [2., 3., 2.], [3., 2., 1.]])
scales = torch.tensor([[1., 2., 1.], [1., 2., 1.], [.5, 3.5, 1.5], [.5, 3.5, 1.5]])
classes = torch.tensor([0.3, 0.33, 0.37])
#print(locs)
#print(class_amount)
print(data_x[0:3:])
x = data_x[0:10]
print(x)
print(bayes_probs(data_x, locs, scales, classes))


tensor([[5.1000, 3.5000, 1.4000, 0.2000],
        [4.9000, 3.0000, 1.4000, 0.2000],
        [4.7000, 3.2000, 1.3000, 0.2000]])
tensor([[5.1000, 3.5000, 1.4000, 0.2000],
        [4.9000, 3.0000, 1.4000, 0.2000],
        [4.7000, 3.2000, 1.3000, 0.2000],
        [4.6000, 3.1000, 1.5000, 0.2000],
        [5.0000, 3.6000, 1.4000, 0.2000],
        [5.4000, 3.9000, 1.7000, 0.4000],
        [4.6000, 3.4000, 1.4000, 0.3000],
        [5.0000, 3.4000, 1.5000, 0.2000],
        [4.4000, 2.9000, 1.4000, 0.2000],
        [4.9000, 3.1000, 1.5000, 0.1000]])
tensor([[4.0402e-05, 7.0209e-05, 9.0586e-05],
        [4.8780e-05, 7.8848e-05, 1.0321e-04],
        [4.9402e-05, 8.3884e-05, 1.5499e-04],
        [5.8724e-05, 8.9560e-05, 1.7932e-04],
        [4.1591e-05, 7.2576e-05, 1.1097e-04],
        [5.6289e-05, 6.0495e-05, 6.0676e-05],
        [7.0897e-05, 8.8229e-05, 2.0573e-04],
        [4.6704e-05, 7.5127e-05, 1.0672e-04],
        [5.8725e-05, 9.4517e-05, 2.0938e-04],
        [3.8169e-05, 7.8572e-05, 1.079

In [261]:
lhat = torch.stack(
        [
            torch.stack(
                calc_probs(locs, softplus(scales), data_x[i], classes)
            )  for i in range(len(data_x))]
        )
print(lhat)

tensor([[4.0402e-05, 7.0209e-05, 9.0586e-05],
        [4.8780e-05, 7.8848e-05, 1.0321e-04],
        [4.9402e-05, 8.3884e-05, 1.5499e-04],
        [5.8724e-05, 8.9560e-05, 1.7932e-04],
        [4.1591e-05, 7.2576e-05, 1.1097e-04],
        [5.6289e-05, 6.0495e-05, 6.0676e-05],
        [7.0897e-05, 8.8229e-05, 2.0573e-04],
        [4.6704e-05, 7.5127e-05, 1.0672e-04],
        [5.8725e-05, 9.4517e-05, 2.0938e-04],
        [3.8169e-05, 7.8572e-05, 1.0793e-04],
        [3.2131e-05, 5.9631e-05, 5.4946e-05],
        [5.4359e-05, 8.2666e-05, 1.5173e-04],
        [3.7949e-05, 8.0929e-05, 1.1823e-04],
        [3.5113e-05, 9.2088e-05, 2.2911e-04],
        [1.5570e-05, 4.3042e-05, 2.2877e-05],
        [2.8309e-05, 4.3987e-05, 3.0475e-05],
        [4.5591e-05, 5.7651e-05, 5.6625e-05],
        [5.3985e-05, 7.1202e-05, 9.2963e-05],
        [3.4154e-05, 5.1368e-05, 3.1718e-05],
        [5.1091e-05, 6.9051e-05, 1.0069e-04],
        [3.8468e-05, 6.3252e-05, 5.2233e-05],
        [7.0555e-05, 7.1142e-05, 1

In [292]:
  
    

def pdf(mean, std, val):
    if std<=0:
        if std<0:
            print('std<0')
        std = 0.0001
    ret = (1 / (std * np.sqrt(2 * np.pi))) * np.exp(-((val - mean) ** 2) / (2 * std ** 2))
    if ret<=0:
        if ret<0:
            print('pdf<0')
        ret = 0.00000000001
    return ret

def calc_probs(values, locs, scales, classes):
    log_prob = []
    for class_id in range(class_amount):
        prob_sum = classes[class_id].clone()
        for atr_i in range(atr_amount):
            tmp = pdf(locs[atr_i, class_id].item(), scales[atr_i, class_id].item(), values[atr_i].item())
            prob_sum *= tmp
        log_prob.append(prob_sum)
    result = log_prob
    return result

def calc_probss(x, sample_locs, sample_scales, sample_class):
    return torch.stack(
        [
            torch.stack(
                calc_probs(sample_locs, softplus(sample_scales), x[i], sample_class)
            )  for i in range(len(data_x))]
        )

def model3(data_x, data_y):
    # sample pk before plate(K) bc reasons
    prior_class = pyro.distributions.Dirichlet(torch.ones(class_amount, ))
    sample_class = pyro.sample('sample_class', prior_class)
    with pyro.plate('class', size=class_amount):
        with pyro.plate('atr', size=atr_amount):
            prior_locs = pyro.distributions.Normal(loc=torch.ones(atr_amount, class_amount) * 3, scale=torch.ones(atr_amount, class_amount) * 2)
            prior_scale = pyro.distributions.Normal(loc=torch.ones(atr_amount, class_amount), scale=torch.ones(atr_amount, class_amount))
            sample_locs = pyro.sample('sample_loc', prior_locs)
            sample_scales = pyro.sample('sample_scale', prior_scale)

    
    #lhat = bayes_probs(data_x, sample_locs, sample_scales, sample_class)
    prob_x = calc_prob(data_x, sample_locs, sample_scales, sample_class)
    #torch.stack(
    #    [
    #        torch.stack(
    #            calc_probs(sample_locs, softplus(sample_scales), data_x[i], sample_class)
    #        )  for i in range(len(data_x))]
    #    )
        
    with pyro.plate('plate', len(data_y)):
        cat = pyro.distributions.Categorical(probs=prob_x)
        pyro.sample("obs", cat, obs=data_y)


def guide3(data_x, data_y):
    param_loc_loc = pyro.param("param_loc_loc", torch.ones(atr_amount, class_amount) * 2.0)
    param_loc_scale = pyro.param("param_loc_scale", torch.ones(atr_amount, class_amount), constraint=constraints.positive)
    param_scale_loc = pyro.param("param_scale_loc", torch.ones(atr_amount, class_amount) * 0.7)
    param_scale_scale = pyro.param("param_scale_scale", torch.ones(atr_amount, class_amount), constraint=constraints.positive)
    
    param_class = pyro.param("param_class", torch.ones(class_amount, ))
    class_prob = pyro.distributions.Dirichlet(param_class)
    sample_class = pyro.sample('sample_class', class_prob)
    with pyro.plate('class', size=class_amount):
        with pyro.plate('atr', size=atr_amount):
            locs = pyro.distributions.Normal(loc=param_loc_loc, scale=param_loc_scale)
            scales = pyro.distributions.Normal(loc=param_scale_loc, scale=param_scale_scale)
            sample_loc = pyro.sample('sample_loc', locs)
            sample_scale = pyro.sample('sample_scale', scales)

    return sample_loc, sample_scale, sample_class

In [297]:
def normalize_class(data_y):
    classes = list(unique_classes)
    print(classes)
    for i in range(len(data_y)):
        data_y[i] = int(classes.index(data_y[i]))
    return data_y

dataset = pd.read_csv('iris.txt', sep=',', header=[0])
data_x = torch.tensor(dataset.iloc[:,0:-1].values)
data_y = dataset.iloc[:,-1]
class_prob = data_y.value_counts(normalize = True)
unique_classes = np.unique(data_y)
class_amount = len(unique_classes)
atr_amount = data_x.shape[1]
data_y = torch.tensor(normalize_class(data_y))

['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [301]:
#mod = model3
#guid = guide3

def train(X, Y):
    pyro.clear_param_store()
    num_iterations=2000
    optim = pyro.optim.Adam({"lr": 0.02})
    svi = pyro.infer.SVI(model3, guide3, optim, loss=pyro.infer.Trace_ELBO())
    losses = list()
    t=tqdm(range(num_iterations))
    for j in t:
        loss = svi.step(X, Y)
        losses.append(loss)
        t.set_postfix(loss=loss)
    return (svi, losses)

train(data_x, data_y)


(<pyro.infer.svi.SVI at 0x7fb9cfb00dd8>,
 [1846.9417358836308,
  762.5208370860937,
  489.3124466767165,
  2147.312649744668,
  2125.0554561752274,
  1310.9654925408822,
  3085.953561617429,
  2892.810029198522,
  268.7498524238258,
  604.9614221941443,
  113.21669667181638,
  1318.190956130163,
  149.73965142489655,
  1835.8962960211622,
  674.6767008971498,
  331.2008539791732,
  907.6720456948468,
  2005.2100324241196,
  2168.623123560364,
  2978.7457221303353,
  840.6761388010823,
  1131.2692082391238,
  991.0072598990363,
  2120.91525800982,
  1369.892622739833,
  940.1688039748368,
  403.79080667539284,
  3462.3006976595243,
  1986.5226990012209,
  1664.2201261732014,
  609.3878089097047,
  283.55734510161966,
  537.684758814859,
  1311.0755269807528,
  137.0467988472702,
  1040.5714782326586,
  381.5766115508766,
  1808.2745663031385,
  394.63509983060686,
  1852.2541291709938,
  2334.452655543749,
  482.45765017422417,
  270.74122476778723,
  1810.692347666729,
  325.9062129036

In [302]:
for name, value in pyro.get_param_store().items():
    print(name, pyro.param(name))
    
for i in range(30):
    params = guide3(None, None)
    prob_y = calc_prob(data_x, params[0], softplus(params[1]), params[2])
    pred_y = torch.argmax(prob_y, dim=1)
    acc = 1.0 - np.count_nonzero((data_y.type(torch.DoubleTensor)-pred_y.type(torch.DoubleTensor)).detach().numpy()) / len(pred_y)
    print(acc)

param_loc_loc tensor([[3.2805, 3.5444, 3.2837],
        [3.3263, 2.7535, 2.4134],
        [1.2766, 3.8677, 4.4291],
        [0.9162, 1.0345, 2.4921]], requires_grad=True)
param_loc_scale tensor([[0.8226, 0.4429, 0.4604],
        [0.9504, 0.7266, 0.8063],
        [0.4100, 0.0890, 1.0188],
        [0.9211, 0.4670, 0.0844]], grad_fn=<AddBackward0>)
param_scale_loc tensor([[ 1.4625,  1.9372,  1.8602],
        [ 1.3115,  1.4119,  1.4214],
        [-0.1284, -0.6544,  1.9859],
        [ 1.4079,  0.8667, -0.7495]], requires_grad=True)
param_scale_scale tensor([[0.4769, 0.4224, 0.4302],
        [0.6370, 0.5534, 0.6167],
        [0.4249, 0.1359, 0.5113],
        [0.5748, 0.6567, 0.1652]], grad_fn=<AddBackward0>)
param_class tensor([3.4124, 5.2808, 4.6809], grad_fn=<AddBackward0>)
0.8523489932885906
0.9597315436241611
0.8657718120805369
0.9798657718120806
0.9395973154362416
0.738255033557047
0.9530201342281879
0.7986577181208054
0.8389261744966443
0.87248322147651
0.9597315436241611
0.82550335570

In [109]:
print(data_y)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1.])


In [283]:
def model3(data_x, data_y):
    # sample pk before plate(K) bc reasons
    prior_class = pyro.distributions.Dirichlet(torch.ones(class_amount, ))
    sample_class = pyro.sample('sample_class', prior_class)
    with pyro.plate('class', size=class_amount):
        with pyro.plate('atr', size=atr_amount):
            prior_locs = pyro.distributions.Normal(loc=torch.ones(atr_amount, class_amount) * 3, scale=torch.ones(atr_amount, class_amount) * 2)
            prior_scale = pyro.distributions.Normal(loc=torch.ones(atr_amount, class_amount), scale=torch.ones(atr_amount, class_amount))
            sample_locs = pyro.sample('sample_loc', prior_locs)
            sample_scales = pyro.sample('sample_scale', prior_scale)

    
    #lhat = bayes_probs(data_x, sample_locs, sample_scales, sample_class)
    lhat = calc_prob(data_x, sample_locs, sample_scales, sample_class)
    #torch.stack(
    #    [
    #        torch.stack(
    #            calc_probs(sample_locs, softplus(sample_scales), data_x[i], sample_class)
    #        )  for i in range(len(data_x))]
    #    )
        
    with pyro.plate('plate', len(data_y)):
        cat = pyro.distributions.Categorical(probs=lhat)
        pyro.sample("obs", cat, obs=data_y)


def guide3(data_x, data_y):
    param_loc_loc = pyro.param("param_loc_loc", torch.ones(atr_amount, class_amount) * 3.3)
    param_loc_scale = pyro.param("param_loc_scale", torch.ones(atr_amount, class_amount) * 0.5, constraint=constraints.positive)
    param_scale_loc = pyro.param("param_scale_loc", torch.ones(atr_amount, class_amount) * 0.7)
    param_scale_scale = pyro.param("param_scale_scale", torch.ones(atr_amount, class_amount) * 0.3, constraint=constraints.positive)
    
    param_class = pyro.param("param_class", torch.ones(class_amount, ), constraint=constraints.interval(0.01, 1000))
    class_prob = pyro.distributions.Dirichlet(param_class)
    sample_class = pyro.sample('sample_class', class_prob)
    with pyro.plate('class', size=class_amount):
        with pyro.plate('atr', size=atr_amount):
            locs = pyro.distributions.Normal(loc=param_loc_loc, scale=param_loc_scale)
            scales = pyro.distributions.Normal(loc=param_scale_loc, scale=param_scale_scale)
            sample_loc = pyro.sample('sample_loc', locs)
            sample_scale = pyro.sample('sample_scale', scales)

    return sample_loc, sample_scale, sample_class

In [181]:
  """return (torch.sum(- 0.5 * 
                        torch.log(2 *
                        np.pi * torch.abs(scales))
                        - (x - locs)**2 / 
                        (scales / 2),
                        dim=-1)
                        + torch.log(pk))
    all_prob = []
    for item in x:
        class_prob = []
        for class_i in range(class_amount):
            buff = pk[class_i].item()
            for atr_i in range(atr_amount):
                var = (scales[atr_i, class_i] ** 2)
                loc = locs[atr_i, class_i]
                log_scale = torch.log(scales[atr_i, class_i])
                #print(type(scales[atr_i, class_i]))
                #x = -((item[atr_i] - loc) ** 2) / (2 * var) - log_scale - math.log(math.sqrt(2 * math.pi))
                x = pyro.distributions.Normal(loc=loc, scale=softplus(scales[atr_i, class_i])).log_prob(item[atr_i])
                #x = pyro.distributions.Normal(loc=locs[atr_i, class_i], scale=softplus(scales[atr_i, class_i])).log_prob(item[atr_i])
                #print(x)
                buff*= math.exp(x)               #buff*=pdf(mean = locs[atr_i, class_i].detach().numpy(), std=softplus(scales[atr_i, class_i]).detach().numpy(), val=item[atr_i].detach().numpy())
            class_prob.append(math.log(buff))
        all_prob.append(class_prob)
    return torch.tensor(all_prob)"""

0.33333333333333337
0.98
0.33333333333333337
0.3733333333333333
0.6333333333333333
0.6666666666666667
0.52
0.7733333333333333
0.8733333333333333
0.6266666666666667
0.42000000000000004
0.8933333333333333
0.64
0.33333333333333337
0.33333333333333337
0.33333333333333337
0.5333333333333333
0.6599999999999999
0.18666666666666665
0.6666666666666667
0.5533333333333333
0.33333333333333337
0.33333333333333337
0.9266666666666666
0.33333333333333337
0.33333333333333337
0.6599999999999999
0.5733333333333333
0.6533333333333333
0.38
